In [1]:
import tensorflow as tf
from tensorflow import keras
import os, inspect
import sys
import pandas as pd
import numpy as np
import joblib

In [2]:
#현재 접근 폴더 경로 가져오기
NOW =os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) #현재 파일이 위치한 경로

if NOW == '/content': # 구글 드라이브일 경우 접근 허용 및 경로 탐색
  from google.colab import drive
  drive.mount('/content/drive')
  finalPath = os.path.join(NOW,'drive/MyDrive/Colab Notebooks/Final') 
  NOW = finalPath 

Mounted at /content/drive


### 모델 불러오기

In [3]:
Fold =10
#ann
for i in range(Fold):
  s1="MODELPATH = os.path.join(NOW,'Models/ann_%d.h5')" %(i+1)# 경로지정
  exec(s1)
  s2="model_%d =keras.models.load_model(MODELPATH)" %(i+1) # 불러오기
  exec(s2)

### 가중치 가져오기

In [6]:
Weight_PATH = os.path.join(NOW, 'weight/Weight.csv')
Weight= pd.read_csv(Weight_ann_PATH,sep=',')

### test 데이터 가져오기

In [8]:
TESTPATH = os.path.join(NOW,'DATA/01_split/test.csv') # 경로지정
test =pd.read_csv(TESTPATH,sep =',') 

In [9]:
ID = test['id'] # id 가져오기
x_test = test.drop('id',axis =1) # input 데이터 가져오기

### 모델로 추론 진행하기 + 앙상블

In [12]:
for i in range(Fold):
  s1= "predict_%d = model_%d.predict(x_test)" %(i+1,i+1)
  exec(s1)
  
# ann 모델 앙상블
y_predict = np.zeros(predict_1.shape)
for i in range(Fold):
  s1 = "y_predict = y_predict + predict_%d*Weight.iloc[i,0]" %(i+1)
  exec(s1)

### 추론된 값으로 클래스 추론하기
Noise  = [1,0,0,0,0], 
In     = [0,1,0,0,0],
Out    = [0,0,1,0,0],
Normal = [0,0,0,1,0],
Other  = [0,0,0,0,1]


In [13]:
Y_Results=[]
for i in range(len(y_predict)):
  temp_results=y_predict[i]
  find=int(np.where(temp_results==temp_results.max())[0][0]) # 제일 확률 높은 곳 찾기
  if   find==0:
    Y_Results.append('noise') # Noise  = [1,0,0,0,0]
  elif find==1:
    Y_Results.append('in') # In = [0,1,0,0,0]
  elif find==2:
    Y_Results.append('out') #Out = [0,0,1,0,0]
  elif find==3:
    Y_Results.append('normal') # Normal = [0,0,0,1,0]
  elif find==4:
    Y_Results.append('other') # Other = [0,0,0,0,1]

Y_Results=pd.DataFrame(Y_Results)
Predict=pd.concat([ID,Y_Results],axis=1)

Predict.columns=['id','leaktype']

### 예측한 데이터 저장하기

In [14]:
PREDICTPATH = os.path.join(NOW,'results/predictions.csv')
Predict.to_csv(PREDICTPATH,index=False)